In [1]:
from google.cloud import run_v2

In [2]:
GOOGLE_CLOUD_PROJECT = "service-calls-dev"
LOCATION = "us-west1"
ARTIFACT_REPO = "task-containers"

In [3]:
def create_job(client: run_v2.JobsClient, job_id: str):
    job = run_v2.Job()

    request = run_v2.CreateJobRequest(
        parent=f"projects/{GOOGLE_CLOUD_PROJECT}/locations/{LOCATION}",
        job=job,
        job_id=job_id,
    )

    op = client.create_job(request=request)
    return op.result()

In [8]:
def update_cloud_run(
    client: run_v2.JobsClient,
    job_id: str,
    bucket_name: str,
    dataset_name: str,
    year: int,
    overwrite: bool = True,
    test: bool = False,
):
    """
    Updates cloud run job config
    """
    # Initialize request argument(s)
    new_envs = [
        run_v2.EnvVar(name="var", value="newbatch")
    ]
    # the below boilerplate was necessary to properly encapsulate "new_envs"
    if "job" in job_id.lower():
        # leaving command blank uses default container entrypoint
        # command = ["run"]
        args = ["run", "--is_test", str(test)]
    else:
        # command=["python3", "cloudrun_elt.py"]
        args = [
            "--bucket_name",
            str(bucket_name),
            "--dataset_name",
            str(dataset_name),
            "--year",
            str(year),
            "--overwrite",
            str(overwrite),
            "--test",
            str(test),
        ]
    container = run_v2.Container(
        image=f"{LOCATION}-docker.pkg.dev/{GOOGLE_CLOUD_PROJECT}/{ARTIFACT_REPO}/agent:test",
        # command=command,
        args=args,
        env=new_envs,
    )
    task_template = run_v2.TaskTemplate(
        containers=[container],
        max_retries=3,
    )
    execution_template = run_v2.ExecutionTemplate(
        task_count=1,
        template=task_template,
    )
    job = run_v2.Job(
        name=f"projects/{GOOGLE_CLOUD_PROJECT}/locations/{LOCATION}/jobs/{job_id}",
        template=execution_template,
    )
    # job.template.template.max_retries = 3
    # job.template.template.containers.env = envs

    # instantiate updatejobrequest
    request = run_v2.UpdateJobRequest(job=job)

    # send the request
    operation = client.update_job(request=request)

    print("Waiting for operation to complete...")

    return operation.result()

In [9]:
client = run_v2.JobsClient()
job_id = "agent"
resp = update_cloud_run(
    client,
    job_id,
    bucket_name="bigger-buck",
    dataset_name="bigger-data",
    year=2011,
)

Waiting for operation to complete...


In [4]:
job_id = "job"  # trigger alternate command/args
resp = update_cloud_run(
    client,
    job_id,
    bucket_name="dbt-bucketer",
    dataset_name="dbt-dataseter",
    year=2011,
)

NameError: name 'client' is not defined

In [20]:
foo = True
str(foo)

'True'

In [23]:
resp

name: "projects/service-calls-dev/locations/us-west1/jobs/agent"
uid: "b097275b-7fd1-4405-956d-191ff789eb22"
generation: 5
create_time {
  seconds: 1686977199
  nanos: 229650000
}
update_time {
  seconds: 1687224412
  nanos: 564605000
}
launch_stage: GA
template {
  task_count: 1
  template {
    containers {
      image: "us-west1-docker.pkg.dev/service-calls-dev/task-containers/agent:test"
      command: "python3"
      command: "main.py"
      args: "--bucket_name"
      args: "my-buck4"
      args: "--dataset_name"
      args: "models4"
      args: "--year"
      args: "2017"
      args: "--overwrite"
      args: "True"
      args: "--test"
      args: "False"
      env {
        name: "var"
        value: "newbatch"
      }
      resources {
        limits {
          key: "memory"
          value: "512Mi"
        }
        limits {
          key: "cpu"
          value: "1000m"
        }
      }
    }
    max_retries: 3
    timeout {
      seconds: 600
    }
    service_account: "

In [24]:
type(resp)

google.cloud.run_v2.types.job.Job

In [29]:
[att for att in resp.__dir__() if "__" not in att]

['observed_generation',
 'generation',
 'update_time',
 'terminal_condition',
 'template',
 'reconciling',
 'conditions',
 'launch_stage',
 'uid',
 'annotations',
 'LabelsEntry',
 'last_modifier',
 'etag',
 'delete_time',
 'labels',
 'expire_time',
 'name',
 'create_time',
 'satisfies_pzs',
 'client',
 'client_version',
 'binary_authorization',
 'latest_created_execution',
 'creator',
 'AnnotationsEntry',
 'execution_count']

In [10]:
resp.template.template

containers {
  image: "us-west1-docker.pkg.dev/service-calls-dev/task-containers/agent:test"
  args: "--bucket_name"
  args: "bigger-buck"
  args: "--dataset_name"
  args: "bigger-data"
  args: "--year"
  args: "2011"
  args: "--overwrite"
  args: "True"
  args: "--test"
  args: "False"
  env {
    name: "var"
    value: "newbatch"
  }
  resources {
    limits {
      key: "memory"
      value: "512Mi"
    }
    limits {
      key: "cpu"
      value: "1000m"
    }
  }
}
max_retries: 3
timeout {
  seconds: 600
}
service_account: "23753207316-compute@developer.gserviceaccount.com"
execution_environment: EXECUTION_ENVIRONMENT_GEN2

In [15]:
# type: proto.marshal.collections.repeated.RepeatedComposite
# list of containers. select the first to get the inside
resp.template.template.containers

[image: "us-west1-docker.pkg.dev/service-calls-dev/task-containers/agent:test"
args: "--bucket_name"
args: "bigger-buck"
args: "--dataset_name"
args: "bigger-data"
args: "--year"
args: "2011"
args: "--overwrite"
args: "True"
args: "--test"
args: "False"
env {
  name: "var"
  value: "newbatch"
}
resources {
  limits {
    key: "memory"
    value: "512Mi"
  }
  limits {
    key: "cpu"
    value: "1000m"
  }
}
]

In [23]:
resp.template.template.containers[0].env[0].name

'var'

In [36]:
[run_v2.EnvVar(name=str(f"VAR{i}"), value=(str(i))) for i in range(3)]

[name: "VAR0"
 value: "0",
 name: "VAR1"
 value: "1",
 name: "VAR2"
 value: "2"]